In [ ]:
# https://en.wikipedia.org/wiki/Big_Five_personality_traits
# https://ipip.ori.org/newBigFive5broadKey.htm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
file_to_open = pd.read_csv("Resources/data.csv", delimiter="\t")
data=file_to_open.copy()
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.isnull().values.any()

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
print(f"Number of participants: {len(data)}")

In [ ]:
data.age.value_counts()

In [ ]:
# calculate maximum year we're looking for in the data
max_year = 2012-13
print(f"Youngest participient's date of birth: {max_year}")

In [ ]:
data= data.loc[data["age"]<2000]
data.head()

In [ ]:
data["age"] = [2012-x if x >1960 else x for x in data["age"]]
data= data.loc[data["age"]<=120]
data.head()

In [ ]:
data.age.sort_values(ascending=False) 

In [ ]:
data.describe()

In [ ]:
# data.dtypes

In [ ]:
data.country.value_counts()

In [ ]:
data.country.value_counts().mean()

In [ ]:
countries = pd.DataFrame(data["country"].value_counts())
c_125= countries.loc[countries["country"]>125]
plt.bar(c_125.index,c_125["country"])
plt.show()

In [ ]:
c_125.nunique()

In [ ]:
# Recheck how many cluster we should have as maximum
from sklearn.preprocessing import MinMaxScaler

df = data.drop("country", axis=1)
trim_df = df.drop(df.columns[:6], axis=1)
columns = list(trim_df.columns)
scaler = MinMaxScaler(feature_range=(0,1))
scale_df = scaler.fit_transform(trim_df)
scale_df = pd.DataFrame(scale_df, columns=columns)
scale_df.head()

In [ ]:
#Elbow Method 
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

kmeans = KMeans()
visualizer = KElbowVisualizer(kmeans, k=(2,15))
visualizer.fit(df)
visualizer.poof()

In [ ]:
# cluster analysis- Aysin

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5)

In [ ]:
kmeans.fit(scale_df)

In [ ]:
predicted_clusters = kmeans.predict(scale_df)

In [ ]:
centers = kmeans.cluster_centers_
labels = kmeans.labels_
labels

In [ ]:
len(labels)

In [ ]:
centers

In [ ]:
# scale_df = scale_df.add(labels)
scale_df['cluster'] = labels.tolist()
scale_df.head()

In [ ]:
# analysis of the clusters- Michael

In [ ]:
scale_df

In [ ]:
wieghted_df = scale_df
wieghted_df

In [ ]:
wieghted_df['E2'] = 1.2 - wieghted_df['E2'] 

In [ ]:
wieghted_df['E4'] = 1.2 -wieghted_df['E4']

In [ ]:
wieghted_df['E6'] = 1.2 - wieghted_df['E6']

In [ ]:
wieghted_df['E8'] = 1.2 - wieghted_df['E8']

In [ ]:
wieghted_df['E10'] = 1.2 - wieghted_df['E10']

In [ ]:
wieghted_df['N2'] = 1.2 - wieghted_df['N2']

In [ ]:
wieghted_df['N4'] = 1.2 - wieghted_df['N4']

In [ ]:
wieghted_df['A1'] = 1.2 - wieghted_df['A1']

In [ ]:
wieghted_df['A3'] = 1.2 - wieghted_df['A3']

In [ ]:
wieghted_df['A5'] = 1.2- wieghted_df['A5']

In [ ]:
wieghted_df['A7'] = 1.2 - wieghted_df['A7']

In [ ]:
wieghted_df['C2'] = 1.2 - wieghted_df['C2']

In [ ]:
wieghted_df['C4'] = 1.2 - wieghted_df['C4']

In [ ]:
wieghted_df['C6'] = 1.2 - wieghted_df['C6']

In [ ]:
wieghted_df['C8'] = 1.2 - wieghted_df['C8']

In [ ]:
wieghted_df['O2'] = 1.2 - wieghted_df['O2']

In [ ]:
wieghted_df['O4'] = 1.2 - wieghted_df['O4']

In [ ]:
wieghted_df['O6'] = 1.2 - wieghted_df['O6']

In [ ]:
wieghted_df

In [ ]:
E = wieghted_df

In [ ]:
E['E_Sum_Result'] = wieghted_df.loc[:,["E1","E2","E3","E4","E5","E6","E7","E8","E9","E10"]].sum(axis=1)
E

In [ ]:
N = wieghted_df
N['N_Sum_Result'] = wieghted_df.loc[:,"N1":"N10"].sum(axis=1)
N

In [ ]:
A = wieghted_df
A['A_Sum_Result'] = wieghted_df.loc[:,"A1":"A10"].sum(axis=1)
A

In [ ]:
C = wieghted_df
C['C_Sum_Result'] = wieghted_df.loc[:,"C1":"C10"].sum(axis=1)
C

In [ ]:
O = wieghted_df
O['O_Sum_Result'] = wieghted_df.loc[:,"O1":"O10"].sum(axis=1)
O

In [ ]:
sums_df = pd.DataFrame()

In [ ]:
sums_df["O"] = O['O_Sum_Result']
sums_df["C"] = C['C_Sum_Result']
sums_df["E"] = E['E_Sum_Result']
sums_df["A"] = A['A_Sum_Result']
sums_df["N"] = N['N_Sum_Result']
sums_df["Cluster"] = wieghted_df['cluster']
sums_df

In [ ]:
cluster_avg = sums_df.groupby('Cluster').mean()
cluster_avg

In [ ]:
# analysis of the questions- Stephen
# mean response of the questions

In [ ]:
# demographic analysis-James
#gender, age

In [ ]:
#demographic analysis -Hai
# country, handedness,engnat